In [89]:
import numpy as np
from dynamic_coding_analysis.analysis_pipeline import get_data
from dynamic_coding_analysis.analysis_pipeline import process_spikes as spk
import matplotlib.pyplot as plt
from matplotlib import gridspec
from scipy.interpolate import interp1d
from scipy.ndimage import gaussian_filter1d
from tqdm import trange

In [90]:
# define sessions
save_folder = 'G:/My Drive/Giocomo Lab/RandomForage/aggregate_data/'

mice = ['Milan', 'Pisa', 'Hanover', 'Boston', 'Camden', 'Calais', 
        'Seattle', 'Portland', 'Juneau', 'Quebec', 'Toronto', 'Vancouver', 
        'Seoul', 'Degu', 'Busan', 'Inchon', 'Ulsan', 
        'Mumbai', 'Kerala', 'Goa', 'Punjab', 'Salvador'
       ]

sessions = ([['0420_2', '0424_1', '0425_1'], 
             ['0428_1', '0429_2', '0430_1', '0501_1', '0502_1'],  
             ['0613_1', '0614_1', '0615_2', '0617_5'], 
             ['0615_1', '0617_1', '0619_2'], 
             ['0708_2', '0710_2', '0711_2', '0713_1'],  
             ['0710_2', '0712_2', '0713_2'], 
             ['1005_1', '1006_1', '1007_1', '1008_1', '1009_1', '1010_1'],
             ['1005_2', '1007_1', '1008_1', '1010_1'], 
             ['1102_1', '1104_1', '1105_1', '1106_1'], 
             ['1007_1', '1009_1'], 
             ['1111_1', '1112_1', '1113_1', '1114_1', '1115_1', '1117_1'], 
             ['1114_1', '1115_1', '1118_1'], 
             ['0720', '0721', '0722', '0724', '0725'], 
             ['0720', '0721', '0722', '0723', '0724', '0725'], 
             ['0730', '0731', '0801', '0802', '0803'], 
             ['0810', '0811', '0812', '0813', '0814', '0815'],
             ['0802'], 
             ['1125_1', '1129_1', '1130_1', '1201_1', '1202_1'], 
             ['1207_1', '1208_1', '1209_1', '1210_2', '1211_1', '1213_1'], 
             ['1207_1', '1209_1', '1210_1', '1211_1'], 
             ['1214_1', '1215_1', '1216_1', '1217_1', '1218_1', '1219_1'], 
             ['1202_1', '1203_1', '1204_1']
])

print('N mice: ' + str(len(mice)))

# make a dict to hold data
data = {}
for session, m in zip(sessions, mice):
    data[m] = {}
    for s in session:
        data[m][s] = {}

N mice: 22


In [91]:
'''
HISTOLOGY PROCESSING:
--------------------
X - ML: 0 is first slice where dot inside the second C is first visible, slices are 65um thick
        L is positive, M is negative
Y - AP: 0 is back of brain, measure in line perpendicular to edge, A is positive
Z - DV: 0 is MEC border, MGC defines with straight line from edge of HPC parallel to brain surface
        MGC calls V from this point negative...might make positive just to keep things clean?

Measure 2 points: (1) where probe enters MEC and (2) where probe tip was (remember to account for retraction!)
'''

'\nHISTOLOGY PROCESSING:\n--------------------\nX - ML: 0 is first slice where dot inside the second C is first visible, slices are 65um thick\n        L is positive, M is negative\nY - AP: 0 is back of brain, measure in line perpendicular to edge, A is positive\nZ - DV: 0 is MEC border, MGC defines with straight line from edge of HPC parallel to brain surface\n        MGC calls V from this point negative...might make positive just to keep things clean?\n\nMeasure 2 points: (1) where probe enters MEC and (2) where probe tip was (remember to account for retraction!)\n'

In [92]:
''' Hanover '''
m = 'Hanover'
# L, DiO (cyan)
s = '0615_2'
d = data[m][s]

# estimate--faint dye and torn slice
convert_tip = 5000/1455 # pts/um
ML_tip = -3*65
AP_tip_pt = np.sqrt(9.3**2 + 103.3**2)
DV_tip_pt = np.sqrt(475.3**2 + 22.6**2)
final_depth_pt = np.sqrt(882.7**2 + 33**2)
tip_coordinates = np.asarray([ML_tip/convert_tip, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert_tip

convert_enter = 5000/1609 # pts/um
ML_enter = 2*65
AP_enter_pt = np.sqrt(33.9**2 + 157.4**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert_enter, AP_enter_pt, DV_enter])*convert_enter

d['histology'] = (tip_coordinates, enter_coordinates)

In [93]:
# R, DiD (red)
s = '0617_5'
d = data[m][s]

convert_tip = 5000/1175 # pts/um
ML_tip = 3*65
AP_tip_pt = np.sqrt(23.3**2 + 15.3**2)
DV_tip_pt = np.sqrt(522**2 + 246.3**2)
final_depth_pt = np.sqrt(594**2 + 275.9**2)
tip_coordinates = np.asarray([ML_tip/convert_tip, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert_tip

convert_enter = 5000/1175 # pts/um
ML_enter = -6*65
AP_enter_pt = np.sqrt(66.9**2 + 145.1**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert_enter, AP_enter_pt, DV_enter])*convert_enter

d['histology'] = (tip_coordinates, enter_coordinates)

In [94]:
m = 'Boston'
# **R, DiD (red)
s = '0617_1'
d = data[m][s]

convert_tip = 5000/1426 # pts/um
ML_tip = 3*65
AP_tip_pt = np.sqrt(11**2 + 26.2**2)
DV_tip_pt = np.sqrt(570**2 + 56.5**2)
final_depth_pt = np.sqrt(691**2 + 68.5**2)
tip_coordinates = np.asarray([ML_tip/convert_tip, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert_tip

convert_enter = 5000/1426 # pts/um
ML_enter = 0*65
AP_enter_pt = np.sqrt(20.8**2 + 123.5**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert_enter, AP_enter_pt, DV_enter])*convert_enter

d['histology'] = (tip_coordinates, enter_coordinates)

In [95]:
# **R, DiO (cyan)
s = '0619_2'
d = data[m][s]

convert_tip = 5000/1434 # pts/um
ML_tip = 3*65
AP_tip_pt = np.sqrt(23.6**2 + 58.7**2)
DV_tip_pt = np.sqrt(604.6**2 + 103.9**2)
final_depth_pt = np.sqrt(683.1**2 + 117.4**2)
tip_coordinates = np.asarray([ML_tip/convert_tip, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert_tip

convert_enter = 5000/1736
ML_enter = -1*65
AP_enter_pt = np.sqrt(29.8**2 + 153.8**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert_enter, AP_enter_pt, DV_enter])*convert_enter

d['histology'] = (tip_coordinates, enter_coordinates)

In [96]:
''' Camden '''
m = 'Camden'
# R, DiI (orange)
s = '0711_2'
d = data[m][s]

convert_tip = 5000/1413 # pts/um
ML_tip = 5*65
AP_tip_pt = np.sqrt(24**2 + 35.5**2)
DV_tip_pt = np.sqrt(615.42**2 + 179.2**2)
final_depth_pt = np.sqrt(801**2 + 229**2)
tip_coordinates = np.asarray([ML_tip/convert_tip, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert_tip

convert_enter = 5000/1413 # pts/um
ML_enter = 0*65
AP_enter_pt = np.sqrt(57.75**2 + 158.75**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert_enter, AP_enter_pt, DV_enter])*convert_enter

d['histology'] = (tip_coordinates, enter_coordinates)

In [97]:
''' Calais '''
m = 'Calais'
# R, DiI (orange)
s = '0713_2'
d = data[m][s]

convert_tip = 5000/1425 # um/pt
ML_tip = -2*65
AP_tip_pt = np.sqrt(6.5**2 + 8**2)
DV_tip_pt = np.sqrt(289.63**2 + 75.16**2) # estimate--slice is damaged
final_depth_pt = np.sqrt(338.25**2 + 87.77**2)
tip_coordinates = np.asarray([ML_tip/convert_tip, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert_tip

convert_enter = 5000/1425 # um/pt
ML_enter = -4*65
AP_enter_pt = np.sqrt(69.3**2 + 105.08**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert_enter, AP_enter_pt, DV_enter])*convert_enter

d['histology'] = (tip_coordinates, enter_coordinates)

In [98]:
# R, DiD (red--v. faint dye)
s = '0712_2'
d = data[m][s]

convert_tip = 5000/1425 # um/pt
ML_tip = -4*65
AP_tip_pt = np.sqrt(7**2 + 9.25**2)
DV_tip_pt = np.sqrt(188.25**2 + 35.67**2) # estimate--slice is damaged
final_depth_pt = np.sqrt(246.75**2 + 46.75**2)
tip_coordinates = np.asarray([ML_tip/convert_tip, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert_tip

convert_enter = 5000/1425 # um/pt
ML_enter = -4*65
AP_enter_pt = np.sqrt(37.51**2 + 57.56**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert_enter, AP_enter_pt, DV_enter])*convert_enter

d['histology'] = (tip_coordinates, enter_coordinates)

In [99]:
''' Milan '''
m = 'Milan'
# L2, DiD (red)
s = '0424_1'
d = data[m][s]

convert_tip = 5000/1233 # pts/um
ML_tip = -1*65
AP_tip_pt = np.sqrt(32**2 + 8.67**2)
DV_tip_pt = np.sqrt(444**2 + 47.33**2)
final_depth_pt = np.sqrt(108**2 + 511.33**2)
tip_coordinates = np.asarray([ML_tip/convert_tip, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert_tip

convert_enter = 5000/2066 # pts/um
ML_enter = -3*65
AP_enter_pt = np.sqrt(56**2 + 118**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert_enter, AP_enter_pt, DV_enter])*convert_enter

d['histology'] = (tip_coordinates, enter_coordinates)

In [100]:
# L3, DiO (cyan)
s = '0425_1'
d = data[m][s]

convert_tip = 5000/1233 # pts/um
ML_tip = -2*65
AP_tip_pt = np.sqrt(15.5**2 + 14**2)
DV_tip_pt = np.sqrt(124.5**2 + 292.6**2)
final_depth_pt = np.sqrt(91**2 + 308.5**2)
tip_coordinates = np.asarray([ML_tip/convert_tip, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert_tip

convert_enter = 5000/1233 # pts/um
ML_enter = -2*65
AP_enter_pt = np.sqrt(81.5**2 + 38.7**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert_enter, AP_enter_pt, DV_enter])*convert_enter

d['histology'] = (tip_coordinates, enter_coordinates)

In [101]:
# R, DiI (orange)
s = '0420_2'
d = data[m][s]
convert = 5000/1261 # pts/um

ML_tip = 2*65
AP_tip_pt = np.sqrt(25.7**2 + 76.7**2)
DV_tip_pt = np.sqrt(403.8**2 + 101.7**2)
final_depth_pt = np.sqrt(612**2 + 104**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 0
AP_enter_pt = np.sqrt(43.3**2 + 115**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 130.          320.74211517 1651.11042946 2461.43422198]
[  0.         487.23873787   0.        ]


In [102]:
''' Pisa '''
m = 'Pisa'

# L, DiD (red)
s = '0428_1'
d = data[m][s]
convert = 5000/1336 # pt/um

ML_tip = 1*65
AP_tip_pt = np.sqrt(6.7**2 + 24.7**2)
DV_tip_pt = np.sqrt(314.3**2 + 7.7**2)
final_depth_pt = np.sqrt(494**2 + 11.5**2) # estimate, slice damaged
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 2*65
AP_enter_pt = np.sqrt(38.4**2 + 75.9**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[  65.           95.78060273 1176.62539882 1849.30328647]
[130.         318.34198401   0.        ]


In [103]:
# L, DiO (cyan)
s = '0429_2'
d = data[m][s]
convert = 5000/1336 # pt/um

ML_tip = 2*65
AP_tip_pt = np.sqrt(10.2**2 + 26.7**2)
DV_tip_pt = np.sqrt(213**2 + 40**2)
final_depth_pt = np.sqrt(391.5**2 + 73.5**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 4*65
AP_enter_pt = np.sqrt(25.8**2 + 78.1**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 130.          106.96851547  811.09029164 1490.79221246]
[260.         307.82612204   0.        ]


In [104]:
m = 'Pisa'

# R, DiO (cyan)
s = '0430_1'
d = data[m][s]
convert = 5000/1454 # pt/um

ML_tip = 5*65
AP_tip_pt = np.sqrt(36.3**2 + 34.3**2)
DV_tip_pt = np.sqrt(599.3**2 + 177.2**2)
final_depth_pt = np.sqrt(740**2 + 211**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = -1*65
AP_enter_pt = np.sqrt(20.2**2 + 129.4**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 325.          171.73922314 2149.06554972 2646.12788714]
[-65.         450.36854009   0.        ]


In [105]:
# R, DiD (red)
s = '0501_1'
d = data[m][s]
# convert = 

ML_tip = 4*65
AP_tip_pt = np.sqrt(60.7**2 + 33**2)
DV_tip_pt = np.sqrt(496.5**2 + 117**2)
final_depth_pt = np.sqrt(655**2 + 143.3**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = -1*65
AP_enter_pt = np.sqrt(20.2**2 + 129.4**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 260.          237.5875102  1754.12398572 2305.68182386]
[-65.         450.36854009   0.        ]


In [106]:
# R, DiI (orange)
s = '0502_1'
d = data[m][s]
# convert = 

ML_tip = 4*65
AP_tip_pt = np.sqrt(38.2**2 + 60.8**2)
DV_tip_pt = np.sqrt(550**2 + 127**2)
final_depth_pt = np.sqrt(718.6**2 + 158.2**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = -1*65
AP_enter_pt = np.sqrt(19.7**2 + 126.2**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 260.          246.92041501 1941.10151293 2530.28836091]
[-65.         439.23089596   0.        ]


In [107]:
''' Rome '''
m = 'Rome'
# **L, DiO (cyan)
s = '0422_1'
d = data[m][s]

convert_tip = 5000/ # pts/um
ML_tip = *65
AP_tip_pt = np.sqrt(**2 + **2)
DV_tip_pt = np.sqrt(**2 + **2)
final_depth_pt = np.sqrt(**2 + **2)
tip_coordinates = np.asarray([ML_tip/convert_tip, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert_tip

convert_enter = 5000/1354 # pts/um
ML_enter = -1*65
AP_enter_pt = np.sqrt(3.36**2 + 60.69**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert_enter, AP_enter_pt, DV_enter])*convert_enter

d['histology'] = (tip_coordinates, enter_coordinates)

SyntaxError: invalid syntax (<ipython-input-107-618910ec5aaa>, line 7)

In [108]:
''' Seoul '''
m = 'Seoul'
# R, DiI (orange)
s = '0720'
d = data[m][s]
convert = 5000/1305 # um/pt

ML_tip = 4*65
AP_tip_pt = np.sqrt(2.9**2 + 5.7**2)
DV_tip_pt = np.sqrt(286.2**2 + 75.6**2)
final_depth_pt = np.nan # slice is damaged, can't measure
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = -1*65
AP_enter_pt = np.sqrt(26.8**2 + 51.5**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 260.           24.50310644 1134.16304058           nan]
[-65.         222.43648014   0.        ]


In [109]:
# R, DiD (red)
s = '0721'
d = data[m][s]
# convert = 5000/

ML_tip = 2*65
AP_tip_pt = 0
DV_tip_pt = np.sqrt(291**2 + 157.5**2)
final_depth_pt = np.nan # slice is damaged, can't measure
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = -1*65
AP_enter_pt = np.sqrt(26.8**2 + 51.5**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 130.            0.         1267.77232342           nan]
[-65.         222.43648014   0.        ]


In [110]:
# R, DiO (cyan) -- these are tentative, dye is not super visible and the DiD is very bright
s = '0722'
d = data[m][s]
# convert = 5000/

ML_tip = 4*65
AP_tip_pt = np.sqrt(2.9**2 + 5.7**2)
DV_tip_pt = np.sqrt(286.2**2 + 75.6**2)
final_depth_pt = np.nan # slice is damaged, can't measure
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = -1*65
AP_enter_pt = np.sqrt(26.8**2 + 51.5**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 260.           24.50310644 1134.16304058           nan]
[-65.         222.43648014   0.        ]


In [111]:
# L, DiD (red) -- can't make out other dye colors /:
s = '0724'
d = data[m][s]
convert = 5000/1338.3 # um/pt

ML_tip = -6*65
AP_tip_pt = np.sqrt(12.8**2 + 25.3**2)
DV_tip_pt = np.sqrt(362.3**2 + 136.1**2)
final_depth_pt = np.sqrt(444**2 + 166.5**2) # approx
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = -4*65 # maybe one slice lateral, but that slice is v torn up and hard to see
AP_enter_pt = np.sqrt(10**2 + 54.5**2) # approx
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[-390.          105.93162735 1445.93897228 1771.62148945]
[-260.          207.01575384    0.        ]


In [112]:
''' Degu '''
m = 'Degu'

# R, DiI (orange)
s = '0720'
d = data[m][s]
convert = 5000/1337.33

ML_tip = -1*65
AP_tip_pt = np.sqrt(14.8**2 + 27.3**2)
DV_tip_pt = np.sqrt(204.1**2 + 46.1**2)
final_depth_pt = np.sqrt(253**2 + 54.5**2) # approx, slice is damaged
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 0*65
AP_enter_pt = np.sqrt(17.2**2 + 86**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[-65.         116.10321772 782.31077996 967.61269427]
[  0.         327.90386679   0.        ]


In [113]:
# R, DiD (red)
s = '0721'
d = data[m][s]
# convert = 5000/1337.33

ML_tip = 1*65
AP_tip_pt = np.sqrt(17.2**2 + 20.5**2)
DV_tip_pt = np.sqrt(283.4**2 + 106.1**2)
final_depth_pt = np.sqrt(406**2 + 146.2**2) # approx, slice is damaged
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 1*65
AP_enter_pt = np.sqrt(52.8**2 + 89.2**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[  65.          100.0495689  1131.39587999 1613.36796872]
[ 65.         387.54676686   0.        ]


In [114]:
# R, DiO (cyan) -- hard to see in tri-color, need to look at single color
s = '0722'
d = data[m][s]
convert = 5000/1343.33

ML_tip = 1*65
AP_tip_pt = np.sqrt(17.3**2 + 22.6**2)
DV_tip_pt = np.sqrt(270.4**2 + 103.1**2)
final_depth_pt = np.sqrt(415**2 + 158.2**2) # approx, slice is damaged
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 0*65
AP_enter_pt = np.sqrt(27.1**2 + 65.9**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[  65.          105.93591053 1077.13150957 1653.09671343]
[  0.         265.21634254   0.        ]


In [115]:
# L, DiI (orange)
s = '0723'
d = data[m][s]
convert = 5000/1332

ML_tip = -5*65
AP_tip_pt = np.sqrt(18.8**2 + 27.7**2)
DV_tip_pt = np.sqrt(71.8**2 + 22.2**2)
final_depth_pt = np.sqrt(273.3**2 + 89.4**2) # approx, slice is damaged
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = -4*65
AP_enter_pt = np.sqrt(16.6**2 + 64.9**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[-325.          125.66556211  282.10851786 1079.39350736]
[-260.         251.4614396    0.       ]


In [116]:
# L, DiO (cyan)
s = '0725'
d = data[m][s]
# convert = 5000/1337.33

ML_tip = -5*65
AP_tip_pt = np.sqrt(31**2 + 55**2)
DV_tip_pt = np.sqrt(295.1**2 + 133.5**2)
final_depth_pt = np.sqrt(538.7**2 + 241.7**2) # approx, slice is damaged
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = -4*65
AP_enter_pt = np.sqrt(34.3**2 + 134.4**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[-325.          236.99240417 1215.81215714 2216.35742254]
[-260.          520.67487377    0.        ]


In [117]:
''' Busan '''
m = 'Busan'
convert_enter = 5000/1327

# L, DiI (orange)
s = '0802'
d = data[m][s]

ML_tip = 0*65
AP_tip_pt = np.sqrt(19.5**2 + 39.5**2)
DV_tip_pt = np.sqrt(218.95**2 + 29.99**2)
final_depth_pt = np.sqrt(437.5**2 + 50**2)
tip_coordinates = np.asarray([ML_tip/convert_tip, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert_tip

ML_enter = 0*65
AP_enter_pt = np.sqrt(79.1**2 + 0**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert_enter, AP_enter_pt, DV_enter])*convert_enter

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[   0.          178.63384705  896.16524615 1785.67669848]
[  0.         298.04069329   0.        ]


In [118]:
# L, DiD (red)
s = '0803'
d = data[m][s]

ML_tip = 0*65
AP_tip_pt = np.sqrt(10.67**2 + 22.45**2)
DV_tip_pt = np.sqrt(236.63**2 + 19.91**2)
final_depth_pt = np.sqrt(481.33**2 + 34.11**2) 
tip_coordinates = np.asarray([ML_tip/convert_tip, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert_tip

ML_enter = 1*65
AP_enter_pt = np.sqrt(21.38**2 + 101.8**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert_enter, AP_enter_pt, DV_enter])*convert_enter

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[   0.          100.79731073  962.96080465 1956.76037477]
[ 65.         391.94003181   0.        ]


In [119]:
# R, DiI (orange)
m = 'Busan'
s = '0730'
d = data[m][s]
convert = 5000/1330

ML_tip = -2*65
AP_tip_pt = np.sqrt(22.3**2 + 47.3**2)
DV_tip_pt = np.sqrt(277.3**2 + 98.7**2)
final_depth_pt = np.sqrt(418.8**2 + 149.8**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = -3*65
AP_enter_pt = np.sqrt(48**2 + 74**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[-130.          196.59102179 1106.54738445 1672.12314636]
[-195.          331.59514393    0.        ]


In [120]:
# R, DiD (red)
s = '0731'
d = data[m][s]
# convert = 5000/1330

ML_tip = -2*65
AP_tip_pt = np.sqrt(11.6**2 + 24.7**2)
DV_tip_pt = np.sqrt(204.1**2 + 52.1**2)
final_depth_pt = np.sqrt(348.7**2 + 85.5**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = -3*65
AP_enter_pt = np.sqrt(45.6**2 + 70**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[-130.          102.58750327  791.89763936 1349.73369609]
[-195.          314.06978948    0.        ]


In [121]:
# R, DiO (cyan)
s = '0801'
d = data[m][s]
# convert = 5000/1330

ML_tip = -5*65
AP_tip_pt = np.sqrt(14.9**2 + 29.2**2)
DV_tip_pt = np.sqrt(173.9**2 + 24.5**2)
final_depth_pt = np.sqrt(331**2 + 48.3**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = -3*65
AP_enter_pt = np.sqrt(55**2 + 90**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[-325.          123.2400554   660.21566978 1257.53925155]
[-195.          396.52299078    0.        ]


In [122]:
''' Inchon -- nice histology images '''
m = 'Inchon'
convert = 5000/1355

# R, DiI (orange)
s = '0810'
d = data[m][s]

ML_tip = -3*65
AP_tip_pt = np.sqrt(14**2 + 16.67**2)
DV_tip_pt = np.sqrt(276.2**2 + 110.65**2)
final_depth_pt = np.sqrt(396.67**2 + 156.67**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = -3*65
AP_enter_pt = np.sqrt(45.62**2 + 87.3**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[-195.           80.32837421 1097.93240674 1573.75889598]
[-195.          363.47283793    0.        ]


In [123]:
# R, DiD (red)
s = '0811'
d = data[m][s]

ML_tip = -4*65
AP_tip_pt = np.sqrt(11**2 + 18**2)
DV_tip_pt = np.sqrt(293.2**2 + 94.3**2)
final_depth_pt = np.sqrt(408.33**2 + 133**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = -4*65
AP_enter_pt = np.sqrt(38.33**2 + 92.25**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[-260.           77.84141369 1136.49979589 1584.66523674]
[-260.          368.62067576    0.        ]


In [124]:
# R, DiO (cyan)
s = '0812'
d = data[m][s]

# estimate--faint dye
ML_tip = 0*65
AP_tip_pt = np.sqrt(57**2 + 48.33**2)
DV_tip_pt = np.sqrt(243.05**2 + 173**2)
final_depth_pt = np.sqrt(333.67**2 + 221.33**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 0*65
AP_enter_pt = np.sqrt(94.11**2 + 94.79**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[   0.          275.76179036 1100.85806929 1477.50219241]
[  0.         492.89054026   0.        ]


In [125]:
convert = 5000/1447
# L, DiD (red)
s = '0814'
d = data[m][s]

ML_tip = -2*65
AP_tip_pt = np.sqrt(10.67**2 + 16.67**2)
DV_tip_pt = np.sqrt(448.41**2 + 243.01**2)
final_depth_pt = np.sqrt(493.33**2 + 266.67**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 0*65
AP_enter_pt = np.sqrt(1.04**2 + 66.97**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[-130.           68.39104083 1762.3527636  1937.7738145 ]
[  0.         231.43771517   0.        ]


In [126]:
''' Ulsan '''
m = 'Ulsan'
convert = 5000/1422

# R, DiI (orange)
s = '0802'
d = data[m][s]

ML_tip = 7*65
AP_tip_pt = np.sqrt(28.67**2 + 31.67**2)
DV_tip_pt = np.sqrt(536.44**2 + 185.73**2)
final_depth_pt = np.sqrt(591**2 + 203.33**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 6*65
AP_enter_pt = np.sqrt(12.92**2 + 98.58**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 455.          150.20929961 1996.0709405  2197.60643854]
[390.         349.58878214   0.        ]


In [127]:
''' Portland '''
m = 'Portland'
convert = 5000/1312

# R, DiI (orange)
s = '1005_2'
d = data[m][s]

ML_tip = 8*65
AP_tip_pt = np.sqrt(14.5**2 + 25**2)
DV_tip_pt = np.sqrt(424.9**2 + 114.62**2)
final_depth_pt = np.sqrt(491**2 + 127.5**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 4*65
AP_enter_pt = np.sqrt(50.55**2 + 99.87**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 520.          110.13983244 1677.1659619  1933.24767747]
[260.         426.57942996   0.        ]


In [128]:
convert = 5000/1356

# L, DiO (cyan) -- bad slice, hard to see
s = '1007_1'
d = data[m][s]

ML_tip = -5*65
AP_tip_pt = np.sqrt(8.5**2 + 10.5**2)
DV_tip_pt = np.sqrt(165.4**2 + 48**2)
final_depth_pt = np.sqrt(322**2 + 91.5*2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = -5*65
AP_enter_pt = np.sqrt(50.5**2 + 20.1**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[-325.           49.81289117  635.04482397 1188.3629654 ]
[-325.          200.41705305    0.        ]


In [129]:
# L, DiI (orange)
s = '1008_1'
d = data[m][s]

ML_tip = -4*65
AP_tip_pt = np.sqrt(25**2 + 46.5**2)
DV_tip_pt = np.sqrt(184.8**2 + 73.1**2)
final_depth_pt = np.sqrt(331**2 + 132.5*2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = -3*65
AP_enter_pt = np.sqrt(12.8**2 + 73.9**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[-260.          194.66966292  732.78985731 1221.97662371]
[-195.          276.54990167    0.        ]


In [130]:
''' Seattle '''
m = 'Seattle'
convert = 5000/1344.7

# L, DiI (orange)
s = '1008_1'
d = data[m][s]

ML_tip = -2*65
AP_tip_pt = np.sqrt(20.6**2 + 24.3**2)
DV_tip_pt = np.sqrt(134.1**2 + 42.6**2)
final_depth_pt = np.sqrt(330.9**2 + 105.2**2) # estimate--slice is damaged
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = -3*65
AP_enter_pt = np.sqrt(19.9**2 + 67.2**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[-130.          118.45285357  523.17928869 1291.06929094]
[-195.         260.5956412    0.       ]


In [131]:
# convert = 1344.7

# L, DiD (red)
s = '1009_1'
d = data[m][s]

ML_tip = 2*65
AP_tip_pt = np.sqrt(23.3**2 + 35.7**2)
DV_tip_pt = np.sqrt(606**2 + 114.6**2)
final_depth_pt = np.sqrt(722.7**2 + 136.7**2) # estimate--slice is damaged
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 0*65
AP_enter_pt = np.sqrt(31.1**2 + 139.9**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 130.          158.51394394 2293.2280654  2734.86617117]
[  0.         532.88877582   0.        ]


In [132]:
# convert = 1344.7

# L, DiO (cyan)
s = '1010_1'
d = data[m][s]

ML_tip = 0*65
AP_tip_pt = np.sqrt(3.8**2 + 10**2)
DV_tip_pt = np.sqrt(352.1**2 + 36.1**2)
final_depth_pt = np.sqrt(466**2 + 34.4**2) # estimate--slice is damaged
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = -2*65
AP_enter_pt = np.sqrt(59.5**2 + 93.4**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[   0.           39.77713727 1316.07714289 1737.44320111]
[-130.          411.77246878    0.        ]


In [133]:
convert = 5000/1291.3
# R, DiI (orange)
s = '1005_1'
d = data[m][s]

ML_tip = 4*65
AP_tip_pt = np.sqrt(12.5**2 + 38**2)
DV_tip_pt = np.sqrt(437.4**2 + 18.2**2)
final_depth_pt = np.sqrt(597.5**2 + 22.5**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 4*65
AP_enter_pt = np.sqrt(39.2**2 + 129.2**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 260.          154.89477611 1695.10757685 2315.19976028]
[260.         522.79040859   0.        ]


In [134]:
# convert = 1344.7

# R, DiD (red)
s = '1006_1'
d = data[m][s]

ML_tip = 2*65
AP_tip_pt = np.sqrt(20.5**2 + 44.2**2)
DV_tip_pt = np.sqrt(387.1**2 + 64.5**2)
final_depth_pt = np.sqrt(510**2 + 85**2) 
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 2*65
AP_enter_pt = np.sqrt(33.9**2 + 131.9**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 130.          188.65709763 1519.54163611 2001.99339842]
[130.         527.32405356   0.        ]


In [135]:
# convert = 1344.7

# R, DiO (cyan)
s = '1007_1'
d = data[m][s]

ML_tip = 0*65
AP_tip_pt = np.sqrt(8**2 + 22**2)
DV_tip_pt = np.sqrt(296.8**2 + 41.2**2)
final_depth_pt = np.sqrt(515.5**2 + 71.5**2) 
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 2*65
AP_enter_pt = np.sqrt(30.1**2 + 117**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[   0.           90.64276242 1160.24906832 2015.15881003]
[130.         467.78366454   0.        ]


In [136]:
''' Toronto '''
m = 'Toronto'

convert = 5000/1353.3
# L, DiI (orange)
s = '1111_1'
d = data[m][s]

ML_tip = -3*65
AP_tip_pt = np.sqrt(8.3**2 + 41.7**2)
DV_tip_pt = np.sqrt(430.3**2 + 17**2)
final_depth_pt = np.sqrt(606.3**2 + 24**2) 
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = -1*65
AP_enter_pt = np.sqrt(9.6**2 + 119.1**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[-195.          157.09006203 1591.05771555 2241.834133  ]
[-65.         441.46262973   0.        ]


In [137]:
# convert = 
# L, DiD (red)
s = '1112_1'
d = data[m][s]

ML_tip = -5*65
AP_tip_pt = np.sqrt(11.6**2 + 46.5**2)
DV_tip_pt = np.sqrt(426.6**2 + 41.5**2)
final_depth_pt = np.sqrt(704.7**2 + 67.7**2) 
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = -2*65
AP_enter_pt = np.sqrt(16**2 + 116.5**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[-325.          177.06733778 1583.58761492 2615.62283021]
[-130.          434.46973974    0.        ]


In [138]:
# convert = 
# L, DiO (cyan)
s = '1113_1'
d = data[m][s]

ML_tip = -1*65
AP_tip_pt = np.sqrt(6.2**2 + 22**2)
DV_tip_pt = np.sqrt(249.1**2 + 8.3**2)
final_depth_pt = np.sqrt(454.5**2 + 8.5**2) 
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 1*65
AP_enter_pt = np.sqrt(2**2 + 82.7**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ -65.           84.44892643  920.85361502 1679.5221902 ]
[ 65.         305.63873591   0.        ]


In [139]:
# NO SCALE BAR
convert = 5000/1337 
# R, DiI (orange)
s = '1114_1'
d = data[m][s]

ML_tip = 5*65
AP_tip_pt = np.sqrt(46**2 + 75.33**2)
DV_tip_pt = np.sqrt(552.52**2 + 270.17**2)
final_depth_pt = np.sqrt(665.33**2 + 325.33**2) 
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 0*65
AP_enter_pt = np.sqrt(63.56**2 + 200**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 325.          330.08386693 2300.0625656  2769.67203029]
[  0.         784.80476094   0.        ]


In [140]:
# R, DiD (red)
s = '1115_1'
d = data[m][s]

ML_tip = 5*65
AP_tip_pt = np.sqrt(44.5**2 + 54**2)
DV_tip_pt = np.sqrt(449.17**2 + 195.16**2)
final_depth_pt = np.sqrt(530.5**2 + 230.5**2) 
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 1*65
AP_enter_pt = np.sqrt(29.24**2 + 176.67**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 325.          261.67991458 1831.47255002 2163.09667299]
[ 65.         669.68345206   0.        ]


In [141]:
# R, DiO (cyan)--very faint
s = '1117_1'
d = data[m][s]

ML_tip = 3*65
AP_tip_pt = np.sqrt(16.67**2 + 13.33**2)
DV_tip_pt = np.sqrt(313.24**2 + 51.46**2)
final_depth_pt = np.sqrt(466.67**2 + 76.67**2) 
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 3*65
AP_enter_pt = np.sqrt(19.8**2 + 119.12**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 195.           79.82149795 1187.13111068 1768.60952288]
[195.         451.58696939   0.        ]


In [142]:
m = 'Vancouver'

convert = 5000/1352
# R, DiI (orange)
s = '1114_1'
d = data[m][s]

ML_tip = 5*65
AP_tip_pt = np.sqrt(24.7**2 + 48.3**2)
DV_tip_pt = np.sqrt(411.3**2 + 122.3**2)
final_depth_pt = np.sqrt(523.3**2 + 151.3**2) # estimate--slice is damaged
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 4*65
AP_enter_pt = np.sqrt(30.8**2 + 116.4**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 325.          200.62588619 1586.90039802 2014.54694701]
[260.         445.28838727   0.        ]


In [143]:
# convert = 
# R, DiD (red)
s = '1115_1'
d = data[m][s]

ML_tip = 5*65
AP_tip_pt = np.sqrt(11.9**2 + 22.7**2)
DV_tip_pt = np.sqrt(388.5**2 + 64.5**2)
final_depth_pt = np.sqrt(530.3**2 + 88.9**2) 
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 4*65
AP_enter_pt = np.sqrt(36.1**2 + 140**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 325.           94.78572661 1456.42696485 1988.53553735]
[260.         534.68722093   0.        ]


In [144]:
convert = 5000/1303
# R, DiO (cyan)
s = '1118_1'
d = data[m][s]

ML_tip = 5*65
AP_tip_pt = np.sqrt(22.7**2 + 40.7**2)
DV_tip_pt = np.sqrt(367.3**2 + 103.4**2)
final_depth_pt = np.sqrt(485.3**2 + 136.7**2) 
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 4*65
AP_enter_pt = np.sqrt(19.4**2 + 119.2**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 325.          178.82716982 1464.22407662 1934.71013779]
[260.         463.42430293   0.        ]


In [145]:
m = 'Quebec'

convert = 5000/1315.4
# R, DiO (cyan)
s = '1007_1'
d = data[m][s]

ML_tip = 2*65
AP_tip_pt = np.sqrt(46**2 + 120**2)
DV_tip_pt = np.sqrt(412**2 + 119**2)
final_depth_pt = np.sqrt(533**2 + 154**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 1*65
AP_enter_pt = np.sqrt(46.7**2 + 133.4**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 130.          488.50004034 1630.08006138 2108.87093812]
[ 65.         537.24368971   0.        ]


In [146]:
# R, DiD (red)
s = '1009_1'
d = data[m][s]

ML_tip = 2*65
AP_tip_pt = np.sqrt(15**2 + 33**2)
DV_tip_pt = np.sqrt(276.4**2 + 0**2)
final_depth_pt = np.sqrt(453**2 + 0**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 1*65
AP_enter_pt = np.sqrt(30.2**2 + 84**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 130.          137.7875092  1050.63098677 1721.90968527]
[ 65.         339.30317194   0.        ]


In [147]:
m = 'Juneau'

convert = 5000/1348.4
# L, DiO (cyan)
s = '1104_1'
d = data[m][s]

ML_tip = -3*65
AP_tip_pt = np.sqrt(30**2 + 56.7**2)
DV_tip_pt = np.sqrt(349.9**2 + 82.6**2)
final_depth_pt = np.sqrt(572.7**2 + 127.3**2) # estimate--slice is damaged
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = -2*65
AP_enter_pt = np.sqrt(57**2 + 141.7**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[-195.          237.86490791 1333.12597394 2175.45823773]
[-130.          566.35532718    0.        ]


In [148]:
convert = 5000/1345
# L, DiD (red)
s = '1102_1'
d = data[m][s]

ML_tip = 7*65
AP_tip_pt = np.sqrt(30**2 + 41.33**2)
DV_tip_pt = np.sqrt(575.67**2 + 207.78**2)
final_depth_pt = np.sqrt(652**2 + 235.33**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 3*65
AP_enter_pt = np.sqrt(44.83**2 + 151.05**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 455.          189.85217461 2275.16727683 2576.83896884]
[195.         585.73290302   0.        ]


In [149]:
# L, DiI (orange)
s = '1105_1'
d = data[m][s]

ML_tip = -3*65
AP_tip_pt = np.sqrt(28.7**2 + 57.3**2)
DV_tip_pt = np.sqrt(288.3**2 + 97.2**2)
final_depth_pt = np.sqrt(482**2 + 159.3**2) # estimate--slice is damaged
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = -2*65
AP_enter_pt = np.sqrt(44.8**2 + 126.5**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[-195.          238.2368916  1131.02053297 1887.14531035]
[-130.          498.87991067    0.        ]


In [150]:
# L, DiD (red)
s = '1106_1'
d = data[m][s]

ML_tip = -2*65
AP_tip_pt = np.sqrt(44**2 + 101**2)
DV_tip_pt = np.sqrt(337.8**2 + 97.2**2)
final_depth_pt = np.sqrt(564.7**2 + 159**2) # estimate--slice is damaged
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = -1*65
AP_enter_pt = np.sqrt(39.6**2 + 145.1**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[-130.          409.54666708 1306.7148755  2180.88311411]
[-65.         559.13264696   0.        ]


In [151]:
m = 'Goa'

convert = 5000/1406
# L, DiI (orange)
s = '1207_1'
d = data[m][s]

ML_tip = 2*65
AP_tip_pt = np.sqrt(25.5**2 + 41.5**2)
DV_tip_pt = np.sqrt(390.5**2 + 84.4**2)
final_depth_pt = np.sqrt(583**2 + 126**2) # estimate--slice is damaged
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 0*65
AP_enter_pt = np.sqrt(12.4**2 + 126.3**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 130.          173.21591576 1420.75647422 2121.12515167]
[  0.         451.30600852   0.        ]


In [152]:
convert = 5000/660
# R, DiD (red)
s = '1209_1'
d = data[m][s]

ML_tip = 9*65
AP_tip_pt = np.sqrt(7.5**2 + 5.5**2)
DV_tip_pt = np.sqrt(311.5**2 + 115.5**2)
final_depth_pt = np.sqrt(319**2 + 118.3**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 4*65
AP_enter_pt = np.sqrt(6.2**2 + 63.9**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 585.           70.45861832 2516.84522199 2577.49373306]
[260.         486.36422638   0.        ]


In [153]:
convert = 5000/660
# R, DiI (orange)
s = '1210_1'
d = data[m][s]

ML_tip = 6*65
AP_tip_pt = np.sqrt(14.3**2 + 34.3**2)
DV_tip_pt = np.sqrt(311.8**2 + 68.8**2)
final_depth_pt = np.sqrt(284.3**2 + 62.8**2) 
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 4*65
AP_enter_pt = np.sqrt(10.7**2 + 53.6**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 390.          281.52681256 2418.94164792 2205.70793572]
[260.         414.07250289   0.        ]


In [154]:
convert = 5000/660
# R, DiO (cyan)
s = '1211_1'
d = data[m][s]

ML_tip = 9*65
AP_tip_pt = np.sqrt(21.8**2 + 18.8**2)
DV_tip_pt = np.sqrt(315.8**2 + 122.9**2)
final_depth_pt = np.sqrt(318.8**2 + 112.5**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 4*65
AP_enter_pt = np.sqrt(6.2**2 + 60.7**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 585.          218.08183736 2567.21008254 2561.11804546]
[260.         462.24104259   0.        ]


In [155]:
m = 'Kerala'

convert = 5000/656
# L, DiI (orange)
s = '1207_1'
d = data[m][s]

ML_tip = 3*65
AP_tip_pt = np.sqrt(15**2 + 17**2)
DV_tip_pt = np.sqrt(238.6**2 + 147.8**2)
final_depth_pt = np.sqrt(319.7**2 + 198**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = -2*65
AP_enter_pt = np.sqrt(37.2**2 + 69.6**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 195.          172.80158611 2139.24152227 2866.21942804]
[-130.          601.50669686    0.        ]


In [156]:
convert = 5000/656
# L, DiD (red)
s = '1208_1'
d = data[m][s]

ML_tip = 4*65
AP_tip_pt = np.sqrt(7**2 + 7.3**2)
DV_tip_pt = np.sqrt(196.8**2 + 112.4**2)
final_depth_pt = np.sqrt(296.7**2 + 169.3**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 0*65
AP_enter_pt = np.sqrt(28.9**2 + 51.3**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 260.           77.08728573 1727.41061335 2603.68999703]
[  0.         448.78343923   0.        ]


In [157]:
convert = 5000/656
# L, DiO (cyan)
s = '1209_1'
d = data[m][s]

ML_tip = 2*65
AP_tip_pt = np.sqrt(3.3**2 + 8**2)
DV_tip_pt = np.sqrt(328**2 + 6**2)
final_depth_pt = np.sqrt(439.3**2 + 8**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = -2*65
AP_enter_pt = np.sqrt(33.8**2 + 63.3**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 130.           65.95961017 2500.41824283 3348.87833172]
[-130.          546.94231867    0.        ]


In [158]:
convert = 5000/663
# R, DiI (orange)
s = '1210_2'
d = data[m][s]

ML_tip = 8*65
AP_tip_pt = np.sqrt(40.3**2 + 44.7**2)
DV_tip_pt = np.sqrt(294.6**2 + 94.7**2)
final_depth_pt = np.sqrt(382.8**2 + 123**2) # estimate--slice is damaged
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 7*65
AP_enter_pt = np.sqrt(25.6**2 + 92.3**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 520.          453.88046389 2333.68539679 3032.24483048]
[455.         722.35590725   0.        ]


In [159]:
convert = 5000/663
# R, DiD (red)
s = '1211_1'
d = data[m][s]

ML_tip = 10*65
AP_tip_pt = np.sqrt(35.2**2 + 24.5**2)
DV_tip_pt = np.sqrt(285.2**2 + 144.8**2)
final_depth_pt = np.sqrt(340**2 + 172.7**2) # estimate--slice is damaged
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 9*65
AP_enter_pt = np.sqrt(16.3**2 + 79.7**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 650.          323.43095325 2412.16603571 2875.91764363]
[585.         613.49727589   0.        ]


In [160]:
convert = 5000/663
# R, DiO (cyan)
s = '1213_1'
d = data[m][s]

ML_tip = 7*65
AP_tip_pt = np.sqrt(20.2**2 + 43.2**2)
DV_tip_pt = np.sqrt(260**2 + 50.7**2)
final_depth_pt = np.sqrt(329.5**2 + 64.5**2) # estimate--slice is damaged
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 7*65
AP_enter_pt = np.sqrt(22.6**2 + 77.7**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 455.          359.64865628 1997.71591989 2532.07865696]
[455.         610.25657801   0.        ]


In [161]:
m = 'Mumbai'

convert = 5000/661
# R, DiI (orange)
s = '1125_1'
d = data[m][s]

ML_tip = 11*65 
AP_tip_pt = np.sqrt(26.3**2 + 28.7**2)
DV_tip_pt = np.sqrt(287**2 + 153.7**2)
final_depth_pt = np.sqrt(267**2 + 143**2) # estimate--slice is damaged
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 5*65
AP_enter_pt = np.sqrt(25.6**2 + 69.1**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 715.          294.46204265 2462.67159974 2291.09542832]
[325.        557.4106445   0.       ]


In [162]:
convert = 5000/661
# R, DiD (red)
s = '1129_1'
d = data[m][s]

ML_tip = 9*65
AP_tip_pt = np.sqrt(19.7**2 + 50.3**2)
DV_tip_pt = np.sqrt(237.4**2 + 74.9**2)
final_depth_pt = np.sqrt(246**2 + 77.7**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 5*65
AP_enter_pt = np.sqrt(21.7**2 + 56.1**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 585.          408.6246703  1883.02021718 1951.43149282]
[325.         454.99730877   0.        ]


In [163]:
convert = 5000/657.5
# L, DiD (red)
s = '1130_1'
d = data[m][s]

ML_tip = -4*65
AP_tip_pt = np.sqrt(5.5**2 + 21.3**2)
DV_tip_pt = np.sqrt(241.4**2 + 156.4**2)
final_depth_pt = np.sqrt(354.2**2 + 229.5**2) # estimate--slice is damaged
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = -3*65
AP_enter_pt = np.sqrt(19.2**2 + 62.4**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[-260.          167.29001005 2187.35197606 3209.52090621]
[-195.          496.47953188    0.        ]


In [164]:
convert = 5000/660.5
# L, DiI (orange)
s = '1201_1'
d = data[m][s]

ML_tip = 1*65
AP_tip_pt = np.sqrt(5.3**2 + 31.3**2)
DV_tip_pt = np.sqrt(216.7**2 + 217.2**2)
final_depth_pt = np.sqrt(253.7**2 + 254.3**2) # estimate--slice is damaged
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 0*65
AP_enter_pt = np.sqrt(15.5**2 + 76.8**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[  65.          240.31454102 2322.58770676 2719.23160781]
[  0.         593.10004738   0.        ]


In [165]:
convert = 5000/657.5
# L, DiO (cyan)
s = '1202_1'
d = data[m][s]

ML_tip = -6*65
AP_tip_pt = np.sqrt(4.5**2 + 4.5**2)
DV_tip_pt = np.sqrt(142.5**2 + 101.2**2)
final_depth_pt = np.sqrt(143.7**2 + 102**2) # estimate--slice is damaged
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = -7*65
AP_enter_pt = np.sqrt(3**2 + 18.3**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[-390.           48.39514092 1329.11767839 1340.08039552]
[-455.          141.02107492    0.        ]


In [166]:
m = 'Punjab'

convert = 5000/660.8
# R, DiI (orange)
s = '1214_1'
d = data[m][s]

ML_tip = 10*65
AP_tip_pt = np.sqrt(42.3**2 + 41.7**2)
DV_tip_pt = np.sqrt(310.3**2 + 141.9**2)
final_depth_pt = np.sqrt(292.1**2 + 133.6**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 6*65
AP_enter_pt = np.sqrt(31.3**2 + 63.9**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 650.          449.44374114 2581.76636362 2430.41014183]
[390.         538.39329761   0.        ]


In [167]:
convert = 5000/660.8
# R, DiD (red)
s = '1215_1'
d = data[m][s]

ML_tip = 10*65
AP_tip_pt = np.sqrt(24.2**2 + 20**2)
DV_tip_pt = np.sqrt(304.9**2 + 124.7**2)
final_depth_pt = np.sqrt(313**2 + 128**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 6*65
AP_enter_pt = np.sqrt(30.9**2 + 58.6**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 650.          237.55224005 2492.54527939 2558.72581841]
[390.         501.26961574   0.        ]


In [168]:
convert = 5000/660.8
# R, DiO (cyan)
s = '1216_1'
d = data[m][s]

ML_tip = 10*65 # estimate--very faint
AP_tip_pt = np.sqrt(20.2**2 + 18.25**2)
DV_tip_pt = np.sqrt(337.6**2 + 188.6**2)
final_depth_pt = np.sqrt(269.5**2 + 150.5**2) 
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 6*65
AP_enter_pt = np.sqrt(29.7**2 + 67.3**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 650.          205.98666638 2926.06561013 2335.61891529]
[390.         556.61382131   0.        ]


In [169]:
convert = 5000/659.3
# L, DiI (orange)
s = '1217_1'
d = data[m][s]

ML_tip = 8*65
AP_tip_pt = np.sqrt(16**2 + 11**2)
DV_tip_pt = np.sqrt(187.7**2 + 166.9**2)
final_depth_pt = np.sqrt(220.9**2 + 196.4**2) # estimate--slice is damaged
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 5*65 # estimate--slice is missing
AP_enter_pt = np.sqrt(7.9**2 + 36**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 520.          147.25077991 1904.83131541 2241.64855384]
[325.         279.51322661   0.        ]


In [170]:
convert = 5000/659.3
# L, DiD (red)
s = '1218_1'
d = data[m][s]

ML_tip = 7*65
AP_tip_pt = np.sqrt(20.4**2 + 0**2)
DV_tip_pt = np.sqrt(316.3**2 + 54.6**2)
final_depth_pt = np.sqrt(326.9**2 + 56.5**2) # estimate--slice is damaged
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 6*65
AP_enter_pt = np.sqrt(8.8**2 + 48**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 455.          154.70954042 2434.23296485 2515.90080741]
[390.         370.08948885   0.        ]


In [171]:
convert = 5000/659.3
# L, DiO (cyan)
s = '1219_1'
d = data[m][s]

ML_tip = 7*65
AP_tip_pt = np.sqrt(2.8**2 + 40.8**2)
DV_tip_pt = np.sqrt(237.3**2 + 30.2**2)
final_depth_pt = np.sqrt(218**2 + 27.8**2) # estimate--slice is damaged
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 6*65
AP_enter_pt = np.sqrt(9.2**2 + 42.2**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 455.          310.14686464 1814.15124909 1666.65721517]
[390.         327.55348709   0.        ]


In [172]:
m = 'Salvador'

convert = 5000/660
# R, DiI (orange)
s = '1202_1'
d = data[m][s]

ML_tip = 10*65
AP_tip_pt = np.sqrt(8.8**2 + 11.8**2)
DV_tip_pt = np.sqrt(285.6**2 + 84.6**2)
final_depth_pt = np.sqrt(312.7**2 + 92.7**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 8*65 # estimate--slice is missing
AP_enter_pt = np.sqrt(19.8**2 + 28.4**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 650.          111.51556324 2256.56526094 2470.84213086]
[520.         262.27881057   0.        ]


In [173]:
convert = 5000/660
# R, DiD (red)
s = '1203_1'
d = data[m][s]

ML_tip = 7*65
AP_tip_pt = np.sqrt(7**2 + 8.8**2)
DV_tip_pt = np.sqrt(241**2 + 114.1**2) 
final_depth_pt = np.sqrt(305**2 + 144.5**2)
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 7*65
AP_enter_pt = np.sqrt(22**2 + 37.3**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 455.           85.18601695 2020.04148668 2556.80695845]
[455.         328.06529312   0.        ]


In [174]:
convert = 5000/660
# R, DiO (cyan)
s = '1204_1'
d = data[m][s]

ML_tip = 11*65
AP_tip_pt = np.sqrt(16.2**2 + 18**2)
DV_tip_pt = np.sqrt(327.4**2 + 57.8**2)
final_depth_pt = np.sqrt(351**2 + 62**2) # estimate--slice is damaged
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = 7*65
AP_enter_pt = np.sqrt(36.2**2 + 59.3**2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

[ 715.          183.45850973 2518.65856261 2700.25548905]
[455.         526.33417426   0.        ]


In [175]:
m = ''

convert = 5000/
# L, DiO (cyan)
s = ''
d = data[m][s]

ML_tip = *65
AP_tip_pt = np.sqrt(**2 + **2)
DV_tip_pt = np.sqrt(**2 + **2)
final_depth_pt = np.sqrt(**2 + **2) # estimate--slice is damaged
tip_coordinates = np.asarray([ML_tip/convert, AP_tip_pt, DV_tip_pt, final_depth_pt])*convert

ML_enter = *65
AP_enter_pt = np.sqrt(**2 + **2)
DV_enter = 0
enter_coordinates = np.asarray([ML_enter/convert, AP_enter_pt, DV_enter])*convert

print(tip_coordinates)
print(enter_coordinates)
d['histology'] = (tip_coordinates, enter_coordinates)

SyntaxError: invalid syntax (<ipython-input-175-11bc382ae3cb>, line 3)

In [176]:
np.save(save_folder + '_histology.npy', data)

In [91]:
data.keys()

dict_keys(['Milan', 'Pisa', 'Hanover', 'Boston', 'Camden', 'Calais', 'Seattle', 'Portland', 'Juneau', 'Quebec', 'Toronto', 'Vancouver', 'Seoul', 'Degu', 'Busan', 'Inchon', 'Ulsan', 'Mumbai', 'Kerala', 'Goa', 'Punjab', 'Salvador'])